<a href="https://colab.research.google.com/github/dubidub/Coursera_Capstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [83]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.'
df = pd.read_html(url, header=0)[0]

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [84]:
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postcode', 'Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)
df['Neighbourhood'] = np.where(
    df['Neighbourhood'] == 'Not assigned', 
    df['Borough'], 
    df['Neighbourhood']
    )

df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [85]:
df.shape

(103, 3)

In [86]:
lat_lng = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [87]:
lat_lng.rename(columns={'Postal Code':'Postcode'}, inplace=True)

df_new = pd.merge(df, lat_lng, on='Postcode')
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [88]:
df_new.shape

(103, 5)

In [89]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

/bin/bash: conda: command not found
/bin/bash: conda: command not found




---



---



# Explore and cluster the neighborhoods in Toronto

## 1. Remove boroughs that does not contain the word Toronto

In [128]:
df_to = df_new[df_new['Borough'].str.contains('Toronto', regex=False)]
df_to

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259


In [129]:
df_to.shape

(39, 5)

## 2. Create map of Toronto

In [130]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [131]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], df_to['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

## 3. Use Foursquare API to extracts nearby venues 

In [0]:
CLIENT_ID = '----' 
CLIENT_SECRET = '----' 
VERSION = '20180605' # Foursquare API version

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [133]:
toronto_venues = getNearbyVenues(names=df_to['Neighbourhood'],
                                   latitudes=df_to['Latitude'],
                                   longitudes=df_to['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [134]:
print(toronto_venues.shape)

(1726, 7)


In [135]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


## 4. Create dummies for K-clustering

In [136]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 241 uniques categories.


In [137]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

toronto_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront


In [138]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,...,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.02,0.0,0.0,0.010000,0.01,0.0,0.02,0.0,0.000000,0.000000,0.0,0.000000,0.020000,0.0,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.02,0.0000,0.01,0.020000,0.000000,0.0,0.01,...,0.020000,0.0,0.0,0.000000,0.000000,0.0,0.010000,0.000000,0.01,0.000000,0.0,0.000000,0.0,0.030000,0.0,0.0,0.02,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.040000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.017857,0.00,0.0,0.00,0.0,0.000000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.0,0.0000,0.00,0.0000,0.00,0.017857,0.000000,0.0,0.00,...,0.035714,0.0,0.0,0.017857,0.000000,0.0,0.000000,0.000000,0.00,0.017857,0.0,0.000000,0.0,0.017857,0.0,0.0,0.00,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.017857,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.041667,0.0,0.041667,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.00,0.0000,0.00,0.083333,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.041667,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.055556,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.00,0.0000,0.00,0.000000,0.055556,0.0,0.00,...,0.000000,0.0,0.0,0.000000,0.055556,0.0,0.055556,0.055556,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0625,0.00,0.0625,0.00,0.000000,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0


## 5. Use 5 clusters

In [139]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
toronto_grouped_clustering.head()


,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.02,0.0,0.0,0.010000,0.01,0.0,0.02,0.0,0.000000,0.000000,0.0,0.000000,0.020000,0.0,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.02,0.0000,0.01,0.020000,0.000000,0.0,0.01,0.01,...,0.020000,0.0,0.0,0.000000,0.000000,0.0,0.010000,0.000000,0.01,0.000000,0.0,0.000000,0.0,0.030000,0.0,0.0,0.02,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.040000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.017857,0.00,0.0,0.00,0.0,0.000000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.0,0.0000,0.00,0.0000,0.00,0.017857,0.000000,0.0,0.00,0.00,...,0.035714,0.0,0.0,0.017857,0.000000,0.0,0.000000,0.000000,0.00,0.017857,0.0,0.000000,0.0,0.017857,0.0,0.0,0.00,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.017857,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.041667,0.0,0.041667,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.00,0.0000,0.00,0.083333,0.000000,0.0,0.00,0.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.041667,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.055556,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.00,0.0000,0.00,0.000000,0.055556,0.0,0.00,0.00,...,0.000000,0.0,0.0,0.000000,0.055556,0.0,0.055556,0.055556,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0625,0.00,0.0625,0.00,0.000000,0.000000,0.0,0.00,0.00,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [140]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [141]:
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_grouped.head()

,Cluster Labels,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,...,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,1,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.02,0.0,0.0,0.010000,0.01,0.0,0.02,0.0,0.000000,0.000000,0.0,0.000000,0.020000,0.0,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.02,0.0000,0.01,0.020000,0.000000,0.0,...,0.020000,0.0,0.0,0.000000,0.000000,0.0,0.010000,0.000000,0.01,0.000000,0.0,0.000000,0.0,0.030000,0.0,0.0,0.02,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.040000,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.017857,0.00,0.0,0.00,0.0,0.000000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.0,0.0000,0.00,0.0000,0.00,0.017857,0.000000,0.0,...,0.035714,0.0,0.0,0.017857,0.000000,0.0,0.000000,0.000000,0.00,0.017857,0.0,0.000000,0.0,0.017857,0.0,0.0,0.00,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.017857,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,1,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.041667,0.0,0.041667,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.00,0.0000,0.00,0.083333,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.041667,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,1,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.055556,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.00,0.0000,0.00,0.000000,0.055556,0.0,...,0.000000,0.0,0.0,0.000000,0.055556,0.0,0.055556,0.055556,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,1,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0625,0.00,0.0625,0.00,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [142]:
toronto_grouped.shape

(39, 243)

## 6. Display clustering results on map

In [0]:
toronto_merged = df_to

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_grouped.set_index('Neighbourhood'), on='Neighbourhood')

In [144]:
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,...,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.021739,0.0,0.021739,0.0,0.00000,0.021739,0.0,0.0,0.00,0.0,0.00,0.065217,0.021739,0.00000,0.0,0.0,0.0,0.0,0.00000,0.021739,0.0,0.0,0.0,0.0,0.00,0.00000,...,0.00000,0.0,0.021739,0.00,0.0,0.0,0.00,0.021739,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.043478,0.0,0.0,0.00,0.00,0.0,0.0,0.00000,0.00,0.00,0.00,0.0,0.00000,0.0,0.021739
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.02439,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.024390,0.02439,0.0,0.0,0.0,0.0,0.02439,0.000000,0.0,0.0,0.0,0.0,0.00,0.02439,...,0.02439,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.02439,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.024390,0.0,0.0,0.00,0.00,0.0,0.0,0.02439,0.00,0.00,0.00,0.0,0.02439,0.0,0.024390
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.010000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.0,0.00,0.010000,0.010000,0.01000,0.0,0.0,0.0,0.0,0.01000,0.000000,0.0,0.0,0.0,0.0,0.02,0.00000,...,0.01000,0.0,0.010000,0.01,0.0,0.0,0.01,0.010000,0.00,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.01000,0.0,0.00,0.0,0.01,0.0,0.01,0.0,0.01,0.020000,0.0,0.0,0.01,0.00,0.0,0.0,0.00000,0.01,0.01,0.01,0.0,0.00000,0.0,0.000000
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.000000,0.0,0.010000,0.0,0.00000,0.010000,0.0,0.0,0.02,0.0,0.01,0.030000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.03000,0.000000,0.0,0.0,0.0,0.0,0.01,0.00000,...,0.02000,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.01,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.01,0.0,0.00,0.0,0.00,0.0,0.02,0.010000,0.0,0.0,0.00,0.00,0.0,0.0,0.01000,0.00,0.00,0.01,0.0,0.00000,0.0,0.000000
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00,0.00000,...,0.00000,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.000000,0.0,0.0,0.00,0.25,0.0,0.0,0.00000,0.00,0.00,0.00,0.0,0.00000,0.0,0.000000


In [147]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters